In [10]:
import cv2
import numpy as np

def is_contour_darker(contour, image, dilation_size=5):
    # Create a mask for the contour
    contour_mask = np.zeros_like(image, dtype=np.uint8)
    cv2.drawContours(contour_mask, [contour], -1, (255, 255, 255), cv2.FILLED)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Dilate the contour mask to include neighboring pixels
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (dilation_size, dilation_size))
    dilated_mask = cv2.dilate(contour_mask[:, :, 0], kernel, iterations=1)

    # Calculate the average intensity value within the contour
    contour_avg_intensity = np.mean(gray[np.where(dilated_mask == 255)])

    # Calculate the average intensity value of the surrounding area
    surrounding_avg_intensity = np.mean(gray[np.where(dilated_mask == 0)])

    # Compare the average intensity values
    if contour_avg_intensity < surrounding_avg_intensity:
        return True
    else:
        return False


# Example usage:
# Load the image
image = cv2.imread('./test4.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to obtain a binary image
_, threshold = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

# Find contours in the binary image
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through the contours
for contour in contours:
    # Detect if the contour is darker than the surrounding area
    is_darker = is_contour_darker(contour, image, dilation_size=10)

    # Draw a bounding rectangle around the contour
    x, y, w, h = cv2.boundingRect(contour)
    color = (0, 255, 0) if is_darker else (0, 0, 255)
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

# Display the image with the contours and their detection results
cv2.imshow('Contour Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
